In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_id = os.getenv("REDDIT_USERNAME")
user_pw = os.getenv("REDDIT_PASSWORD")

In [ ]:
import praw
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

reddit = praw.Reddit(
    client_id = client_id,
    client_secret = client_secret,
    username = user_id,
    password = user_pw,
    user_agent = "script:Scraper:v1.0"
)

In [ ]:
# List of keywords to search for
keywords = ['solar', 'wind', 'hydro']
comments_data = []

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
subreddit = reddit.subreddit("RenewableEnergy")

for keyword in keywords:
  for submission in subreddit.search(keyword, sort='new', limit=25): # Get 25 of the newest posts
    submission.comments.replace_more(limit=0) # Remove "more comments" instance
    for comment in submission.comments.list()[:50]: # Get first 50 comments per post
        # sentiment_score = analyzer.polarity_scores(comment.body)['compound'] # Get the compound score
        sentiment_score = analyzer.polarity_scores(comment.body)
        if sentiment_score['compound'] >= 0.05:
          sentiment = "positive"
        elif sentiment_score['compound'] <= -0.05:
          sentiment = "negative"
        else:
          sentiment = "neutral"
        comments_data.append({
            "keyword": keyword,
            "post_title": submission.title,
            "comment_body": comment.body,
            "comment_score": comment.score,
            "compound": sentiment_score['compound'],
            "pos": sentiment_score['pos'],
            "neu": sentiment_score['neu'],
            "neg": sentiment_score['neg'],
            "sentiment": sentiment,
        })

In [ ]:
# Create dataframe
df = pd.DataFrame(comments_data)
# Save as csv
df.to_csv("redditcomments_fullscore.csv", index=False, encoding='utf-8')
print("Data collection saved as csv file.")